In [1]:
#check the direction

In [2]:
import os
print(os.getcwd())

C:\Program Files\Microsoft VS Code


In [3]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"H:\Regional_Water_Company\For_WR_Report\New_Geo_Data")

# Verify the change
print(os.getcwd())

H:\Regional_Water_Company\For_WR_Report\New_Geo_Data


In [4]:
import pandas as pd

# Read the Excel file
Geo_Data = pd.read_excel("New_Merge_Data_dd.xlsx")
# Show the first 5 rows
print(Geo_Data.head())

                 point_name  Depth_1  Depth_2 Type_Geo Elevation  Total_Depth  \
0  abasabad tokhmanjir asma      0.0      3.0       GC      1057         26.6   
1  abasabad tokhmanjir asma      3.0      6.6       GP      1057         26.6   
2  abasabad tokhmanjir asma      6.6      9.4       CL      1057         26.6   
3  abasabad tokhmanjir asma      9.4     14.0       GP      1057         26.6   
4  abasabad tokhmanjir asma     14.0     15.5       CL      1057         26.6   

    Lon_D    Lat_D  Thickness_meters  Thickness_Percent  
0  59.429  36.5557               3.0          11.278195  
1  59.429  36.5557               3.6          13.533835  
2  59.429  36.5557               2.8          10.526316  
3  59.429  36.5557               4.6          17.293233  
4  59.429  36.5557               1.5           5.639098  


In [7]:
import pandas as pd

# Read the geological data
Geo_Data = pd.read_excel("New_Merge_Data_dd.xlsx")

# Define the geological layer abbreviations, full names, and categories
layer_definitions = {
    'GC': {'full_name': 'Gravelly Clay', 'category': 'Fine-grained'},
    'GP': {'full_name': 'Poorly Graded Gravel', 'category': 'Coarse-grained'},
    'CL': {'full_name': 'Clay (Low Plasticity)', 'category': 'Fine-grained'},
    'Undifind': {'full_name': 'Undefined', 'category': 'Uncategorized'},
    'GW': {'full_name': 'Well-Graded Gravel', 'category': 'Coarse-grained'},
    'SM': {'full_name': 'Silty Sand', 'category': 'Coarse-grained'},
    'CL-ML': {'full_name': 'Clay-Silt Mixture', 'category': 'Fine-grained'},
    'BEDROCK': {'full_name': 'Bedrock', 'category': 'Uncategorized'},
    'SP': {'full_name': 'Poorly Graded Sand', 'category': 'Coarse-grained'},
    'SW': {'full_name': 'Well-Graded Sand', 'category': 'Coarse-grained'},
    'ML': {'full_name': 'Silt (Low Plasticity)', 'category': 'Fine-grained'},
    'OL': {'full_name': 'Organic Silt/Clay', 'category': 'Fine-grained'},
    'SC': {'full_name': 'Clayey Sand', 'category': 'Coarse-grained'},
    'CCL': {'full_name': 'Clay (Low Plasticity)', 'category': 'Fine-grained'},
    'CH': {'full_name': 'Clay (High Plasticity)', 'category': 'Fine-grained'},
    'GM': {'full_name': 'Gravelly Silt', 'category': 'Fine-grained'},
    'GP-GM': {'full_name': 'Gravel-Silt Mixture', 'category': 'Medium'},
    'SC-SM': {'full_name': 'Sand-Silt Mixture', 'category': 'Medium'},
    '<Null>': {'full_name': 'Missing Data', 'category': 'Uncategorized'}
}

# Function to print layer definitions by category
def print_layer_categories():
    print("Geological Layer Classifications:\n")
    for category in ['Coarse-grained', 'Fine-grained', 'Medium', 'Uncategorized']:
        print(f"{category} Layers:")
        for abbr, info in layer_definitions.items():
            if info['category'] == category:
                print(f"  {abbr}: {info['full_name']}")
        print()

# Print the categorized layers
print_layer_categories()

# Add full names and categories to the DataFrame
Geo_Data['Full_Name'] = Geo_Data['Type_Geo'].map(lambda x: layer_definitions.get(x, {'full_name': 'Unknown'})['full_name'])
Geo_Data['Category'] = Geo_Data['Type_Geo'].map(lambda x: layer_definitions.get(x, {'category': 'Uncategorized'})['category'])

# Calculate thickness sum per point and add validation columns
Geo_Data['Thickness_Sum'] = Geo_Data.groupby('point_name')['Thickness_Percent'].transform('sum')
Geo_Data['Thickness_Difference'] = Geo_Data['Thickness_Sum'] - 100
Geo_Data['Thickness_Status'] = Geo_Data['Thickness_Sum'].apply(
    lambda x: 1 if x > 100 else (-1 if x < 100 else 0)
)

# Analysis: Count unique points per category
point_category_counts = Geo_Data.groupby('Category')['point_name'].nunique()
print("\nNumber of Unique Points per Category:")
for category, count in point_category_counts.items():
    print(f"  {category}: {count}")

# Analysis: Thickness_Status distribution
thickness_status_counts = Geo_Data.groupby('Thickness_Status')['point_name'].nunique()
print("\nThickness Status Distribution (Unique Points):")
print(f"  Sum < 100 (Status -1): {thickness_status_counts.get(-1, 0)}")
print(f"  Sum = 100 (Status 0): {thickness_status_counts.get(0, 0)}")
print(f"  Sum > 100 (Status 1): {thickness_status_counts.get(1, 0)}")

# Analysis: Thickness_Percent statistics by category
thickness_stats = Geo_Data.groupby('Category')['Thickness_Percent'].describe()
print("\nThickness Percent Statistics by Category:")
print(thickness_stats)

# Filter out points where the absolute Thickness_Difference is greater than 1
Filtered_Geo_Data = Geo_Data[Geo_Data['Thickness_Difference'].abs() <= 1]
filtered_points = Filtered_Geo_Data['point_name'].nunique()

# Save the complete and filtered DataFrames to Excel
Geo_Data.to_excel("New_Merge_Data_with_Layers.xlsx", index=False)
Filtered_Geo_Data.to_excel("New_Merge_Data_with_Layers_Filtered.xlsx", index=False)

# Display a summary of the data
print("\nSample of Updated DataFrame (Original):")
print(Geo_Data[['point_name', 'Type_Geo', 'Full_Name', 'Category', 'Thickness_Percent', 
                'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status']].head())
print("\nSample of Filtered DataFrame (|Thickness_Difference| <= 1):")
print(Filtered_Geo_Data[['point_name', 'Type_Geo', 'Full_Name', 'Category', 'Thickness_Percent', 
                        'Thickness_Sum', 'Thickness_Difference', 'Thickness_Status']].head())

# Analysis: Number of points filtered out
original_points = Geo_Data['point_name'].nunique()
print(f"\nPoints Analysis:")
print(f"  Total unique points in original data: {original_points}")
print(f"  Total unique points in filtered data: {filtered_points}")
print(f"  Points removed (|Thickness_Difference| > 1): {original_points - filtered_points}")

Geological Layer Classifications:

Coarse-grained Layers:
  GP: Poorly Graded Gravel
  GW: Well-Graded Gravel
  SM: Silty Sand
  SP: Poorly Graded Sand
  SW: Well-Graded Sand
  SC: Clayey Sand

Fine-grained Layers:
  GC: Gravelly Clay
  CL: Clay (Low Plasticity)
  CL-ML: Clay-Silt Mixture
  ML: Silt (Low Plasticity)
  OL: Organic Silt/Clay
  CCL: Clay (Low Plasticity)
  CH: Clay (High Plasticity)
  GM: Gravelly Silt

Medium Layers:
  GP-GM: Gravel-Silt Mixture
  SC-SM: Sand-Silt Mixture

Uncategorized Layers:
  Undifind: Undefined
  BEDROCK: Bedrock
  <Null>: Missing Data


Number of Unique Points per Category:
  Coarse-grained: 381
  Fine-grained: 446
  Medium: 3
  Uncategorized: 249

Thickness Status Distribution (Unique Points):
  Sum < 100 (Status -1): 25
  Sum = 100 (Status 0): 431
  Sum > 100 (Status 1): 18

Thickness Percent Statistics by Category:
                 count       mean        std       min       25%        50%  \
Category                                             